In [33]:
import tensorcircuit as tc
import tensorflow as tf
import math
import numpy as np

K = tc.set_backend('tensorflow')

edges = []

# n, m = map(int, input().split())
# for _ in range(m):
#     edges.append([int(i) for i in input().split()])

n, m = 8, 12
edges = [
    [0,1],[0,2],[0,3],[1,4],[1,5],[2,6],[2,7],[3,4],[3,5],[4,7],[5,6],[6,7]
]
p = 3

# p = int(input().split()[0])

# n,m=4,3
# edges.append((0,1));
# edges.append((1,2));
# edges.append((2,3));
# edges.append((3,0));
# p=4;

In [34]:

def u_beta(c, beta):
    for i in range(n):
        c.rx(i, theta = beta)

def u_gamma(c, gamma):
    for e in edges:
        x, y = e
        c.rzz(x, y, theta = gamma)

def loss_cell(params):
    betas, gammas = params[:p],params[p:]
    c = tc.Circuit(n);
    for i in range(n):
        c.h(i)
    for i in range(p):
        u_gamma(c, gammas[i])
        u_beta(c, betas[i])
    return c

def loss(params):
    c = loss_cell(params)
    loss = K.real(0.);
    for e in edges:
        loss += K.real(c.expectation_ps(z = [e[0], e[1]]))
    return loss

QAOA_vvag = K.jit(tc.backend.vvag(loss))

params = K.implicit_randn(
    shape=[1, 2 * p], stddev=0.1
)
opt = K.optimizer(tf.keras.optimizers.SGD())

for i in range(1000):
    loss, grads = QAOA_vvag(params)
    if i % 100 == 0 :
        tmp_params = params.numpy().tolist()
        for t in range(2 * p) :
            tmp_params[0][t] += math.pi * 3 / 2
#             if tmp_params[0][t] > math.pi :
#                tmp_params[0][t] -= math.pi
        print(tmp_params)
    params = opt.update(grads, params)
loss, grads = QAOA_vvag(params)
print(min(loss))

[[4.857576298709732, 4.514707419272286, 4.657146341349703, 4.724521543081623, 4.790457579489571, 4.765718436982256]]
[[4.996131169792038, 3.8737838149031223, 4.197405743594986, 4.491411375995499, 5.2955788850744785, 5.08812938928209]]
[[4.946610558029038, 3.7644159078558506, 4.186473536487442, 4.405095535512787, 5.42152177095018, 5.10381205677591]]
[[4.939825671907288, 3.7416381716688694, 4.179859447475296, 4.385219591852051, 5.442963349815232, 5.108308273549897]]
[[4.939402866359574, 3.7402276873549045, 4.179382848735672, 4.3839534103830875, 5.444242107864243, 5.108577179904801]]
[[4.939380127187592, 3.7401521086653293, 4.179357039924485, 4.383886027332169, 5.444311070438248, 5.108591067787033]]
[[4.939378890391213, 3.7401481151541294, 4.179355669017655, 4.383882123227936, 5.444314229484421, 5.10859202146135]]
[[4.939378771181923, 3.7401478171309055, 4.1793557286222995, 4.383881855007035, 5.444314706321579, 5.108592200275284]]
[[4.939378830786568, 3.7401478171309055, 4.179355669017655

In [ ]:
[[2.960954653220721, 0.11959540098905563, 3.0826746245198926, 2.9844858964257917, 0.09620922803878784, 0.1445174366235733, 0.06461828947067261, 2.9874743094020566, 0.00924196932464838, 3.1042861250513276], [0.0878397598862648, 0.11266443878412247, 0.05856814607977867, -0.09404543787240982, -0.11809418350458145, -0.11837350577116013, 0.058744799345731735, 0.09243162721395493, -0.12965407967567444, 0.16312158107757568], [-0.08910016715526581, -0.07977859675884247, -0.07939491420984268, -0.07153329253196716, 0.13102684915065765, -0.11115294694900513, 0.07448526471853256, -0.06576144695281982, 0.04844211786985397, 0.01349711138755083], [0.23618720471858978, -0.11162898689508438, 0.051019031554460526, -0.11425477266311646, -0.03683150187134743, 0.12355871498584747, -0.040558747947216034, 0.1562386006116867, -0.005125925410538912, 0.07801534980535507], [0.11379218101501465, -0.12561288475990295, -0.02383531630039215, 0.09284789115190506, -0.012544400058686733, -0.047384217381477356, -0.1426527202129364, 0.041656769812107086, 0.09560687839984894, -0.11901910603046417], [-0.04840712994337082, 0.1268976330757141, -0.09396905452013016, 0.13900229334831238, 0.05905917286872864, -0.04198264703154564, -0.030375365167856216, -0.08623218536376953, -0.04612124711275101, 0.23541758954524994], [-0.09564964473247528, 0.09543540328741074, -0.059947412461042404, 0.08210336416959763, -0.00634548906236887, -0.07455120235681534, 0.13548244535923004, 0.01675364188849926, 0.004980591591447592, 0.10074788331985474], [0.18554972112178802, 0.025229258462786674, -0.13966643810272217, -0.12559017539024353, -0.0461956262588501, 0.1332557052373886, 0.045870598405599594, 0.10014678537845612, 0.252797394990921, -0.028401628136634827]]
[[2.9509727527671537, 0.10960546880960464, 3.072775812941142, 2.9745542873912534, 0.08654839545488358, 0.15450944006443024, 0.07449661195278168, 2.997451441484042, 0.019148467108607292, 3.094328636425086], [0.09781521558761597, 0.12257827818393707, 0.04897123947739601, -0.084098681807518, -0.1280381679534912, -0.12823589146137238, 0.06830392777919769, 0.10241236537694931, -0.11966396868228912, 0.17309637367725372], [-0.07912573963403702, -0.07033077627420425, -0.06942792981863022, -0.061674702912569046, 0.14079654216766357, -0.10116411000490189, 0.08444518595933914, -0.06486613303422928, 0.03848971053957939, 0.003509224858134985], [0.2262098491191864, -0.12158212810754776, 0.04102669283747673, -0.12424708902835846, -0.046824678778648376, 0.11362703889608383, -0.030573569238185883, 0.16579391062259674, 0.004722684621810913, 0.07613059133291245], [0.12370476871728897, -0.1156214028596878, -0.013847608119249344, 0.10276288539171219, -0.002554565202444792, -0.05736643821001053, -0.14188414812088013, 0.03169836103916168, 0.0856349915266037, -0.11314541101455688], [-0.03857582435011864, 0.13684052228927612, -0.08398366719484329, 0.1489924192428589, 0.05806317925453186, -0.0519668385386467, -0.0403657928109169, -0.09615348279476166, -0.056101053953170776, 0.22552816569805145], [-0.08568884432315826, 0.08552183210849762, -0.0698917955160141, 0.07215956598520279, -0.01633533276617527, -0.07868007570505142, 0.12551066279411316, 0.01304219476878643, -0.004965352825820446, 0.1077653020620346], [0.17557786405086517, 0.015246175229549408, -0.14965666830539703, -0.13555055856704712, -0.05616733804345131, 0.12326209247112274, 0.03590944781899452, 0.09020064026117325, 0.2428298145532608, -0.03789597004652023]]
[[2.9409930427842816, 0.09961634129285812, 3.0628332515531262, 2.964590923982211, 0.07665640860795975, 0.16450224816799164, 0.08443329483270645, 3.007429840164729, 0.029062699526548386, 3.0843989980691155], [0.10779037326574326, 0.13248199224472046, 0.03926520049571991, -0.0741882249712944, -0.1380002498626709, -0.13810361921787262, 0.07781390100717545, 0.11239262670278549, -0.1096755787730217, 0.1830686628818512], [-0.06915614753961563, -0.06717733293771744, -0.059473782777786255, -0.05183158069849014, 0.15064430236816406, -0.09118612855672836, 0.09429945051670074, -0.0744023323059082, 0.02852092869579792, -0.0064793536439538], [0.21623949706554413, -0.13153117895126343, 0.03103513829410076, -0.13423889875411987, -0.05681679770350456, 0.103712297976017, -0.02059043012559414, 0.174544095993042, 0.014290357939898968, 0.0702272430062294], [0.1336342990398407, -0.10562968999147415, -0.0038579613901674747, 0.112729012966156, 0.00743674673140049, -0.06735541671514511, -0.15150469541549683, 0.02172750234603882, 0.07566022127866745, -0.11429185420274734], [-0.02871580235660076, 0.14679524302482605, -0.0739976018667221, 0.15898238122463226, 0.06784667074680328, -0.06195070594549179, -0.05035461485385895, -0.10605405271053314, -0.06606846302747726, 0.21564485132694244], [-0.07573423534631729, 0.07563411444425583, -0.07983023673295975, 0.062242358922958374, -0.02632433921098709, -0.06976202130317688, 0.11558506637811661, 0.022202221676707268, -0.014870626851916313, 0.11667796224355698], [0.16561554372310638, 0.005270593333989382, -0.15964365005493164, -0.1455170214176178, -0.06614328175783157, 0.11327198892831802, 0.02599903754889965, 0.08031344413757324, 0.23286744952201843, -0.04733981564640999]]
[[2.9310174753242215, 0.08962912112474442, 3.052877040701457, 2.954618664579936, 0.0667208805680275, 0.17449457943439484, 0.094381183385849, 3.0174054225259503, 0.03896409273147583, 3.0745320861511907], [0.11776447296142578, 0.142372265458107, 0.029503697529435158, -0.06435997784137726, -0.1479703187942505, -0.14798739552497864, 0.08706403523683548, 0.12237071990966797, -0.09969010204076767, 0.19303598999977112], [-0.059194415807724, -0.07581855356693268, -0.04953613132238388, -0.041985515505075455, 0.16054654121398926, -0.08123184740543365, 0.10346833616495132, -0.08430387079715729, 0.018543891608715057, -0.0164683498442173], [0.20627960562705994, -0.14147455990314484, 0.021044772118330002, -0.1442296952009201, -0.06680742651224136, 0.093818299472332, -0.010610385797917843, 0.17868514358997345, 0.022050565108656883, 0.06261403113603592], [0.1435721516609192, -0.09563805162906647, 0.006132674403488636, 0.12270963937044144, 0.017428798601031303, -0.07734647393226624, -0.16138049960136414, 0.011755724437534809, 0.06568866223096848, -0.11889256536960602], [-0.018843350932002068, 0.15675468742847443, -0.06401251256465912, 0.16897059977054596, 0.07779006659984589, -0.07193060219287872, -0.06033913791179657, -0.11587312072515488, -0.07599720358848572, 0.20578427612781525], [-0.06579805910587311, 0.06575216352939606, -0.08978080004453659, 0.05232919752597809, -0.03631362318992615, -0.05992615967988968, 0.10608628392219543, 0.03205472603440285, -0.024666957557201385, 0.12605932354927063], [0.15566793084144592, -0.004691995680332184, -0.16962513327598572, -0.15548665821552277, -0.07612061500549316, 0.10329145938158035, 0.01688537560403347, 0.07107070088386536, 0.22292137145996094, -0.05651019513607025]]
[[2.9210491200261792, 0.07964562624692917, 3.0429177080565175, 2.944645034270831, 0.056773070245981216, 0.18448489904403687, 0.10432334244251251, 3.0273719151788434, 0.04882197827100754, 3.06482473959262], [0.12773670256137848, 0.15224577486515045, 0.01971464976668358, -0.05476035922765732, -0.1579439342021942, -0.15789146721363068, 0.09527301788330078, 0.13234448432922363, -0.08970955014228821, 0.202994704246521], [-0.04924524948000908, -0.08550886064767838, -0.03961752727627754, -0.032125163823366165, 0.17047959566116333, -0.07134757190942764, 0.10352107137441635, -0.0942581370472908, 0.008562377654016018, -0.026457425206899643], [0.19633597135543823, -0.15141019225120544, 0.01105616707354784, -0.15421882271766663, -0.07679623365402222, 0.083992600440979, -0.0006325200665742159, 0.17801953852176666, 0.025570232421159744, 0.05473611503839493], [0.15351378917694092, -0.0856470912694931, 0.01612328551709652, 0.1326950490474701, 0.027420492842793465, -0.0873374342918396, -0.17127183079719543, 0.0017929007299244404, 0.05573030188679695, -0.12249834090471268], [-0.008966099470853806, 0.16671504080295563, -0.054030388593673706, 0.1789543777704239, 0.08775400370359421, -0.08189655095338821, -0.07031051069498062, -0.12493545562028885, -0.08561393618583679, 0.19601298868656158], [-0.055893559008836746, 0.05585966259241104, -0.09974276274442673, 0.042408473789691925, -0.046303387731313705, -0.04999847337603569, 0.10867248475551605, 0.041986312717199326, -0.03410012647509575, 0.13559937477111816], [0.14574392139911652, -0.01463649608194828, -0.1795988827943802, -0.1654549241065979, -0.08609572798013687, 0.09333483129739761, 0.021231912076473236, 0.07063601911067963, 0.21300938725471497, -0.06497759371995926]]
[[2.9110934754186353, 0.06966952234506607, 3.0329621901088437, 2.934675471978732, 0.04682600870728493, 0.19447000324726105, 0.11423881351947784, 3.0373137166076383, 0.058557894080877304, 3.055578353005], [0.1377062052488327, 0.16210447251796722, 0.009927035309374332, -0.0458749383687973, -0.1679181009531021, -0.1678110659122467, 0.0993879958987236, 0.14231134951114655, -0.07973703742027283, 0.21293999254703522], [-0.03931673243641853, -0.09524427354335785, -0.02969355881214142, -0.022212916985154152, 0.18044014275074005, -0.06176494061946869, 0.09489381313323975, -0.10422809422016144, -0.0014208875363692641, -0.03644575923681259], [0.18641890585422516, -0.16133494675159454, 0.0010700271232053638, -0.16420556604862213, -0.08678267896175385, 0.07441799342632294, 0.009343968704342842, 0.17780859768390656, 0.026904383674263954, 0.047239769250154495], [0.16345664858818054, -0.07565752416849136, 0.026112936437129974, 0.1426818072795868, 0.03741062432527542, -0.09732636064291, -0.18109965324401855, -0.008134987205266953, 0.045817773789167404, -0.12184548377990723], [0.0009106541983783245, 0.17667318880558014, -0.04405451938509941, 0.1889282464981079, 0.0977182686328888, -0.09181908518075943, -0.08023761957883835, -0.12378929555416107, -0.08712863177061081, 0.18651191890239716], [-0.046047452837228775, 0.0459267795085907, -0.10971769690513611, 0.0324595682322979, -0.05629407614469528, -0.04004913941025734, 0.11826296895742416, 0.05193588137626648, -0.04225745052099228, 0.14509274065494537], [0.1358615905046463, -0.024562345817685127, -0.18956393003463745, -0.1754128485918045, -0.09606235474348068, 0.08341115713119507, 0.029551994055509567, 0.07342492043972015, 0.20313671231269836, -0.07250597327947617]]
[[2.9011607487970075, 0.05970975384116173, 3.0230194499068936, 2.924718292551585, 0.0368930920958519, 0.20443947613239288, 0.12406707555055618, 3.047170551615306, 0.06789658963680267, 3.047698887186595], [0.1476716697216034, 0.1719609498977661, 0.0001983828842639923, -0.03847561776638031, -0.17788991332054138, -0.17773249745368958, 0.09880189597606659, 0.15226975083351135, -0.06977593153715134, 0.22286875545978546], [-0.029421696439385414, -0.1049371138215065, -0.01975846104323864, -0.012269388884305954, 0.19041430950164795, -0.054133057594299316, 0.08552150428295135, -0.11420337110757828, -0.011403854005038738, -0.046432118862867355], [0.17654623091220856, -0.17124487459659576, -0.008912622928619385, -0.17418891191482544, -0.09676572680473328, 0.06600580364465714, 0.019318178296089172, 0.17841720581054688, 0.026902224868535995, 0.040451809763908386], [0.17339955270290375, -0.06567035615444183, 0.036100417375564575, 0.1526678502559662, 0.047397542744874954, -0.10731009393930435, -0.19040322303771973, -0.01782635971903801, 0.036203935742378235, -0.11674043536186218], [0.010771604254841805, 0.18662196397781372, -0.03409179672598839, 0.1988828182220459, 0.10766623914241791, -0.10163906216621399, -0.09003046900033951, -0.11667420715093613, -0.07985512167215347, 0.17747536301612854], [-0.03632284328341484, 0.03597267344594002, -0.1196991503238678, 0.022494053468108177, -0.06628511846065521, -0.030096296221017838, 0.12809164822101593, 0.061886876821517944, -0.04742511361837387, 0.1542786806821823], [0.1260574758052826, -0.03445981442928314, -0.19951513409614563, -0.18535137176513672, -0.10601268708705902, 0.07354962080717087, 0.03832123428583145, 0.0760619267821312, 0.1933341920375824, -0.07828997075557709]]
[[2.8912716825776776, 0.04979505017399788, 3.013109194236346, 2.9147960563474378, 0.027007268741726875, 0.2143295407295227, 0.13348564505577087, 3.0564740245514592, 0.07583099603652954, 3.042721191542693], [0.15763138234615326, 0.18181730806827545, -0.00939417164772749, -0.03315773606300354, -0.1878567934036255, -0.18764320015907288, 0.09764127433300018, 0.16221891343593597, -0.05982958897948265, 0.23277872800827026], [-0.019566040486097336, -0.11448530107736588, -0.009812185540795326, -0.0023088185116648674, 0.20039518177509308, -0.05085737258195877, 0.07633499056100845, -0.12417859584093094, -0.021383831277489662, -0.056413907557725906], [0.1667376607656479, -0.18113915622234344, -0.01889011822640896, -0.1841670423746109, -0.10674352198839188, 0.062359459698200226, 0.0292875524610281, 0.17794597148895264, 0.02473391965031624, 0.03457711264491081], [0.18334147334098816, -0.05568704754114151, 0.04608387500047684, 0.16265089809894562, 0.05737893655896187, -0.11728285998106003, -0.19161295890808105, -0.021790195256471634, 0.03176229074597359, -0.10972213000059128], [0.020593713968992233, 0.19654671847820282, -0.02416147291660309, 0.20878669619560242, 0.11756560951471329, -0.11083664745092392, -0.09852196276187897, -0.10862654447555542, -0.07135274261236191, 0.16930469870567322], [-0.02689625695347786, 0.026008207350969315, -0.12968362867832184, 0.012519595213234425, -0.07627580314874649, -0.020152663812041283, 0.13794691860675812, 0.07182569056749344, -0.05083811283111572, 0.1619516760110855], [0.1164112240076065, -0.04430856183171272, -0.20944111049175262, -0.19525158405303955, -0.11592888087034225, 0.06382341682910919, 0.0471639558672905, 0.078239805996418, 0.18368901312351227, -0.08093228191137314]]
[[2.88148150046403, 0.040080778300762177, 3.0033011754327497, 2.9050071259313306, 0.017351996153593063, 0.2216491997241974, 0.14023344218730927, 3.059891739981719, 0.08162999153137207, 3.04055758764083], [0.16758321225643158, 0.19166384637355804, -0.01876373030245304, -0.030735988169908524, -0.19781632721424103, -0.19753414392471313, 0.09799206256866455, 0.17215792834758759, -0.049901533871889114, 0.2426658272743225], [-0.009758254513144493, -0.12376675009727478, 0.0001415539300069213, 0.007660445757210255, 0.21037907898426056, -0.04603511095046997, 0.06990480422973633, -0.13414891064167023, -0.03135623037815094, -0.06638534367084503], [0.15699322521686554, -0.19102263450622559, -0.028859220445156097, -0.19413647055625916, -0.11671249568462372, 0.06387700140476227, 0.039245955646038055, 0.174359530210495, 0.019966112449765205, 0.029827997088432312], [0.19327645003795624, -0.04571013152599335, 0.056060634553432465, 0.17262768745422363, 0.06735319644212723, -0.12724363803863525, -0.18732789158821106, -0.017991160973906517, 0.033122383058071136, -0.10310357809066772], [0.03031895123422146, 0.20640254020690918, -0.014328398741781712, 0.21852345764636993, 0.127266064286232, -0.11621959507465363, -0.09824565798044205, -0.10130426287651062, -0.06341131776571274, 0.16193264722824097], [-0.018416710197925568, 0.016038326546549797, -0.1396692991256714, 0.002540762536227703, -0.08626525849103928, -0.010239498689770699, 0.14776325225830078, 0.08172564953565598, -0.05520929768681526, 0.16297084093093872], [0.10710649937391281, -0.0540655292570591, -0.21931515634059906, -0.20506174862384796, -0.12575754523277283, 0.05440034344792366, 0.055813733488321304, 0.07965385913848877, 0.17449143528938293, -0.07946284860372543]]
[[2.871996374922343, 0.031883977353572845, 2.993890421586581, 2.8958325554900846, 0.010151459835469723, 0.2157588005065918, 0.14505989849567413, 3.059491017954894, 0.08897563815116882, 3.03944272746379], [0.17752450704574585, 0.2014867663383484, -0.027826780453324318, -0.03183075040578842, -0.20776553452014923, -0.20739687979221344, 0.10052822530269623, 0.18208618462085724, -0.03999420255422592, 0.2525249421596527], [-4.395481664687395e-05, -0.13308271765708923, 0.010093025863170624, 0.017631109803915024, 0.2203630805015564, -0.04012702777981758, 0.07197247445583344, -0.14411243796348572, -0.04131703078746796, -0.07633855938911438], [0.14732222259044647, -0.2008901834487915, -0.03881249204277992, -0.20408949255943298, -0.1266648918390274, 0.06646965444087982, 0.04918079078197479, 0.16871066391468048, 0.014220794662833214, 0.027092549949884415], [0.2031959891319275, -0.03574516624212265, 0.0660250261425972, 0.18259260058403015, 0.07731563597917557, -0.13718578219413757, -0.18330897390842438, -0.012941581197082996, 0.03435191512107849, -0.098256416618824], [0.039776165038347244, 0.21602636575698853, -0.004878655541688204, 0.22745880484580994, 0.1351047158241272, -0.11716409772634506, -0.09391129016876221, -0.09947230666875839, -0.060608286410570145, 0.15441657602787018], [-0.013711520470678806, 0.006065881345421076, -0.14965476095676422, -0.007439648732542992, -0.09625248610973358, -0.00041443941881880164, 0.15742236375808716, 0.09149045497179031, -0.0608116015791893, 0.1570127308368683], [0.09855407476425171, -0.06363094598054886, -0.22906294465065002, -0.21458718180656433, -0.13526669144630432, 0.04557599127292633, 0.0638183206319809, 0.07993316650390625, 0.1671185940504074, -0.07450158894062042]]